In [12]:
import orientationpy
import numpy
import tifffile
import napari
import matplotlib
import numpy as np
import napari.layers
from napari.types import LayerDataTuple
from typing import List
from magicgui import magicgui

In [13]:
# # Skeleton image

# image = np.load('/home/wittwer/data/skeleton_for_orientationpy.npy')
# image = image.astype(float)
# rx, ry, rz = image.shape

# image.shape

In [14]:
# Sam's data

image = tifffile.imread('/home/wittwer/data/sam/sam-197081.tif')
image = image[:, 180:320, 180:320]
image = image.astype(float) / 256
rx, ry, rz = image.shape

image.shape

(30, 140, 140)

In [8]:
# image = tifffile.imread('/home/wittwer/code/clones/orientationpy/data/3D/3dfib.tif')
# # image = image[:50, :50, :50]
# image = image.astype(float) / 256
# rx, ry, rz = image.shape

# image.shape

In [9]:
# image = tifffile.imread('/home/wittwer/code/clones/orientationpy/data/3D/big-bin4.tif')
# # image = image[:50, :50, :50]
# image = image.astype(float) / 256
# rx, ry, rz = image.shape

# image.shape

In [15]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(image, interpolation='nearest')

<Image layer 'image' at 0x7f530dbbe220>

In [7]:
SIGMA = 1.5

gx, gy, gz = orientationpy.computeGradient(
    image, 
    # mode='gaussian',
    mode='splines',
    # mode='finite_difference',
)

structureTensor = orientationpy.computeStructureTensor((gx, gy, gz), sigma=SIGMA)

# Should energy be None (but still returned) if computeEnergy is False?
dico = orientationpy.computeOrientation(
    structureTensor, 
    mode="fibre", 
    # mode="membrane",
    computeEnergy=True, 
    computeCoherency=True,
    # nProcesses=3
)

theta = dico['theta']
phi = dico['phi']
energy = dico['energy']
coherency = dico['coherency']

imDisplayHSV = numpy.zeros((rx, ry, rz, 3), dtype="f4")

# Hue is the orientation (nice circular mapping)
imDisplayHSV[..., 0] = phi / 360

# Saturation is verticality, so white = up (z)
imDisplayHSV[..., 1] = numpy.sin(numpy.deg2rad(theta))

# Value is original image
imDisplayHSV[..., 2] = image / image.max()

imDisplayRGB = matplotlib.colors.hsv_to_rgb(imDisplayHSV)

# # viewer.add_image(energy)
# # viewer.add_image(coherency)
# viewer.add_image(
#     imDisplayRGB,
#     rgb=True,
#     # channel_axis=-1
# )

  0%|          | 0/30 [00:00<?, ?it/s]

In [10]:
# sc = viewer.screenshot(canvas_only=True, flash=False)

# from PIL import Image
# Image.fromarray(sc)

In [16]:
def compute_orientation(image: np.ndarray, sigma: float) -> np.ndarray:

    gx, gy, gz = orientationpy.computeGradient(
        image, 
        # mode='gaussian',
        mode='splines',
        # mode='finite_difference',
    )

    print(gx.shape, gy.shape)

    structureTensor = orientationpy.computeStructureTensor((gx, gy, gz), sigma=sigma)

    # Should energy be None (but still returned) if computeEnergy is False?
    dico = orientationpy.computeOrientation(
        structureTensor, 
        # mode="fibre", 
        # mode="membrane",
        computeEnergy=True, 
        computeCoherency=True,
        # nProcesses=2
    )

    theta = dico['theta']
    phi = dico['phi']
    energy = dico['energy']
    coherency = dico['coherency']

    rx, ry, rz = image.shape
    imDisplayHSV = numpy.zeros((rx, ry, rz, 3), dtype="f4")

    # Hue is the orientation (nice circular mapping)
    imDisplayHSV[..., 0] = phi / 360

    # Saturation is verticality, so white = up (z)
    imDisplayHSV[..., 1] = numpy.sin(numpy.deg2rad(theta))

    # Value is original image
    imDisplayHSV[..., 2] = image / image.max()

    imDisplayRGB = matplotlib.colors.hsv_to_rgb(imDisplayHSV)

    return imDisplayRGB

@magicgui(call_button='Run', sigma={'widget_type': 'FloatSpinBox', 'min': 0, 'step': 0.1})
def orientation_widget(image_layer: napari.layers.Image, sigma: float=1.0) -> List[LayerDataTuple]:
    if image_layer is None:
        return

    image = image_layer.data

    print(image.shape)
    print('-----------')

    imDisplayRGB = compute_orientation(image, sigma=sigma)

    red, green, blue = np.rollaxis(imDisplayRGB, axis=-1)

    return [
        # (imDisplayRGB, {'name': 'orientation', 'rgb': True}, 'image'),
        (red, {'name': 'ori_red', 'colormap': 'red', 'blending': 'additive'}, 'image'),
        (green, {'name': 'ori_green', 'colormap': 'green', 'blending': 'additive'}, 'image'),
        (blue, {'name': 'ori_blue', 'colormap': 'blue', 'blending': 'additive'}, 'image'),
    ]

viewer.window.add_dock_widget(orientation_widget, name='Orientationpy')

(128, 128, 128)
-----------
(128, 128, 128) (128, 128, 128)


  0%|          | 0/128 [00:00<?, ?it/s]

In [17]:
boxSizePixels = 3

structureTensorBoxes = orientationpy.computeGradientStructureTensorBoxes(
    image,
    [boxSizePixels, boxSizePixels, boxSizePixels],
)

dicoBoxes = orientationpy.computeOrientation(
    structureTensorBoxes,
    mode="fiber",
    # mode='membrane',
    computeEnergy=True,
    computeCoherency=True,
)

thetaBoxes = dicoBoxes['theta']
phiBoxes = dicoBoxes['phi']
energyBoxes = dicoBoxes['energy']
coherencyBoxes = dicoBoxes['coherency']

boxVectorsZYX = orientationpy.anglesToVectors(dicoBoxes)

boxCentresX, boxCentresY, boxCentresZ = numpy.mgrid[
    boxSizePixels // 2 : thetaBoxes.shape[0] * boxSizePixels + boxSizePixels // 2 : boxSizePixels,
    boxSizePixels // 2 : thetaBoxes.shape[1] * boxSizePixels + boxSizePixels // 2 : boxSizePixels,
    boxSizePixels // 2 : thetaBoxes.shape[2] * boxSizePixels + boxSizePixels // 2 : boxSizePixels,
]

boxCentres = np.concatenate((boxCentresX[None], boxCentresY[None], boxCentresZ[None]), axis=0)

_, bx, by, bz = boxCentres.shape

bc = boxCentres.reshape((3, bx*by*bz))
bv = boxVectorsZYX.reshape((3, bx*by*bz))

bv *= energyBoxes.reshape((bx*by*bz)) / energyBoxes.max()
bv *= boxSizePixels

vectors = np.concatenate((bc[None], bv[None]), axis=0)
vectors = np.rollaxis(vectors, axis=2)

viewer.add_vectors(  # (N, 2, 3)
    vectors, edge_width=0.4, name='sliced vectors', edge_color='blue'
)

Computing gradients...done
Computing structure tensor...

100%|██████████| 10/10 [00:00<00:00, 13.99it/s]

done


  0%|          | 0/10 [00:00<?, ?it/s]

<Vectors layer 'sliced vectors' at 0x7f530dab5a60>